In [36]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? 
Nothing done.


In [21]:
import warnings
warnings.filterwarnings('ignore')

from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from skimage.io import imread, imshow, imread_collection, concatenate_images
import cv2
from tqdm import tqdm
import copy
import logging
import os
from pathlib import Path
import const

ROOT_DIR = Path().resolve()
ROOT_DIR = ROOT_DIR / const.sample3
HORIZONTAL_SPLIT = const.HORIZONTAL_SPLIT
VERTICAL_SPLIT = const.VERTICAL_SPLIT
margin = const.margin
if ROOT_DIR == Path().resolve() / const.sample1:
    SCALE = 15/339
elif ROOT_DIR == Path().resolve() / const.sample2:
    SCALE = 35/371
    # 35/339
else: # '3'
    SCALE = 35/310
    # 35/298

size_min = const.size_min
size_max = const.size_max
size_min *= (1/SCALE)*(1/SCALE)
size_max *= (1/SCALE)*(1/SCALE)

if ROOT_DIR == Path().resolve() / const.sample1:
    interval_y = 88
    interval_x = 76
elif ROOT_DIR == Path().resolve() / const.sample2:
    interval_y = 70
    interval_x = 87
elif ROOT_DIR == Path().resolve() / const.sample3:
    interval_y = 75
    interval_x = 74

def s(img, title=''):
    """
    Show image with title.
    :param img: input image
    :param img: title should be shown(default is empty)
    """
    plt.figure(figsize=(6,6))
    plt.title(title + str(img.shape))
    imshow(img)
    plt.show()

def inputPicture(pic_name):
    """
    Concatenate all colored pictures into one picture
    :param pic0~3: input images
    :return: integrated picture
    """
    path = ROOT_DIR / 'for_pin' / pic_name
    pic_color = cv2.imread(str(path), cv2.IMREAD_COLOR)
    pic_color = cv2.cvtColor(pic_color, cv2.COLOR_BGR2RGB)
    return pic_color

def trimming(pic, pic_rec):
    pic_trim = pic[pic_rec[0][1]:pic_rec[1][1], pic_rec[0][0]:pic_rec[2][0], :]
    pic_trim = cv2.resize(pic_trim, (X_width, Y_width))
    return pic_trim

def colorSegmentation(output):
    num_color = [0, 0, 0, 0]
            # [0:red, 1:green, 2:blue, 3:other]
    label = 0

    for i in range(0, output.shape[0]):
        for j in range(0, output.shape[1]):
            if (output[i, j, :] == 0).all() == False:
                if ((output[i, j, 0] > 0) & (output[i, j, 1] == 0) & (output[i, j, 2] == 0)):  #red
                    num_color[0] += 1
                elif ((output[i, j, 0] == 0) & (output[i, j, 1] > 0) & (output[i, j, 2] == 0)):  #green
                    num_color[1] += 1
                elif ((output[i, j, 0] == 0) & (output[i, j, 1] == 0) & (output[i, j, 2] > 0)):  #blue
                    num_color[2] += 1
                else :   #other
                    num_color[3] += 1
    label = np.array(num_color).argmax()
    return label

def is_ferrite_bainite(pic, coordinate): # 0 is on ferrite(blue), 1 is on bainite(not blue)
    FB = 1
    if list(pic[coordinate[1], coordinate[0]]) == [0,0,255]:
        FB = 0
    return FB

def make_margin(pic):
    """
    added 5pixel margin
    :param pic: load picture
    :return: pic that added 5pixel margin
    """
    pic_mat = np.zeros([pic.shape[0] + 2*margin, pic.shape[1] + 2*margin] + [3], dtype=np.uint8) + 255
    pic_mat[margin:-margin, margin:-margin, :] = pic
    return pic_mat

def padding_position(x, y, w, h, p):
    return x-p, y-p, w+p*2, h+p*2

*input all files needed*

In [38]:
im = inputPicture('im_trim.png')
orien = inputPicture('orien_trim.png')
strain = inputPicture('strain_trim.png')
strain4 = inputPicture('strain4_trim.png')
ferrite_bainite = inputPicture('ferrite_bainite_trim.png')

In [39]:
print('im: ', im.shape, 'orien: ', orien.shape, 'strain: ', strain.shape, 'strain4: ', strain4.shape, 
      'ferrite_bainite: ', ferrite_bainite.shape)

im:  (982, 850, 3) orien:  (982, 850, 3) strain:  (982, 850, 3) strain4:  (982, 850, 3) ferrite_bainite:  (982, 850, 3)


*change circumference into white by margin*

In [40]:
im = im[margin:margin+interval_y*(HORIZONTAL_SPLIT-1), 
                                             margin:margin+interval_x*(VERTICAL_SPLIT-1), :]
ferrite_bainite = ferrite_bainite[margin:margin+interval_y*(HORIZONTAL_SPLIT-1), 
                                             margin:margin+interval_x*(VERTICAL_SPLIT-1), :]
im = make_margin(im)
orien = make_margin(orien)
strain = make_margin(strain)
strain4 = make_margin(strain4)
ferrite_bainite = make_margin(ferrite_bainite)
print('im: ', im.shape, 'orien: ', orien.shape, 'strain: ', strain.shape, 'strain4: ', strain4.shape,
      'ferrite_bainite: ', ferrite_bainite.shape)

im:  (978, 846, 3) orien:  (992, 860, 3) strain:  (992, 860, 3) strain4:  (992, 860, 3) ferrite_bainite:  (978, 846, 3)


*from here all features are extracted*<br>
*first of all find contours for each grain, fixing min/max size of grain*

In [41]:
im_gray = im[:,:,0].copy()
print('find contours from this pic:', im_gray.shape)
temp, contours, hierarchy = cv2.findContours(im_gray, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
print('will ditect the grain with')
print('more than {0:.2f} pixcel'.format(size_min))
print('less than {0:.2f} pixcel'.format(size_max))
contours_large = list(filter(lambda c: (cv2.contourArea(c) > size_min) & 
                             (cv2.contourArea(c) < size_max), contours))

find contours from this pic: (978, 846)
will ditect the grain with
more than 153.23 pixcel
less than 56183.60 pixcel


In [42]:
detect_count = 0
roundnesses = []
areas = []
labels_orien = []
labels_strain = []
aspectos = []
tilts = []
f_or_b = []

overall_contour = im.copy()
overall_ellipse = im.copy()
overall_inner_color = im.copy()
overall_nearest_grain = im.copy()

for rect in tqdm(contours_large):

    x, y, w, h = cv2.boundingRect(rect)
    x, y, w, h = padding_position(x, y, w, h, 3)
    M = cv2.moments(rect)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    im_colored = im.copy()
    cv2.drawContours(im_colored, [rect], -1, (0, 0, 255), thickness=-1)
    cv2.drawContours(overall_contour, [rect], -1, (0, 0, 255), thickness=1)
    cv2.drawContours(strain4, [rect], -1, (0, 0, 255), thickness=-1)
    ##cv2.drawContours(overall_ellipse, [rect], -1, (0, 0, 255), thickness=1)
    cv2.drawContours(overall_inner_color, [rect], -1, (0, 0, 255), thickness=-1)
    cv2.drawMarker(overall_nearest_grain, (cx, cy), (0, 0, 255), markerSize=10)
    ellipse = cv2.fitEllipse(rect)
    tilt = round(ellipse[2], 3)
    detect_count_str = str(detect_count) + '.png'
    cv2.imwrite(str(ROOT_DIR / 'output' / 'mat'/ detect_count_str), im_colored)
    cv2.ellipse(overall_ellipse, ellipse, (0, 0, 255), 1)
    cv2.imwrite(str(ROOT_DIR / 'output' / 'snippet' / detect_count_str), overall_contour[y:y + h, x:x + w])
    detect_count = detect_count + 1
    
    area = cv2.contourArea(rect)
    perimeter = cv2.arcLength(rect, True)
    roundness = round(4.0 * np.pi * area / (perimeter * perimeter), 3)  # 1 is perfect circle

    areas.append(area)
    roundnesses.append(roundness)
    labels_orien.append(colorSegmentation(orien[y:y + h, x:x + w]))
    labels_strain.append(colorSegmentation(strain[y:y+h, x:x + w]))
    aspectos.append(round(ellipse[1][0]/ellipse[1][1], 3))  #Major axis/Minor axis, 1 is perfect circle
    if tilt > 90:
        tilt = 180 - tilt
    tilts.append(tilt)
    f_or_b.append(is_ferrite_bainite(ferrite_bainite, (cx, cy)))

cv2.imwrite(str(ROOT_DIR / 'output' / 'overall_nearest_grain.png'), overall_nearest_grain)
cv2.imwrite(str(ROOT_DIR / 'output' / 'overall_contour.png'), overall_contour)
cv2.imwrite(str(ROOT_DIR / 'output' / 'overall_ellipse.png'), overall_ellipse)
cv2.imwrite(str(ROOT_DIR / 'output' / 'overall_inner_color.png'), overall_inner_color)

100%|██████████| 239/239 [00:12<00:00, 19.06it/s]


True

*calculate the nearest particle distance*

In [43]:
# fill center with the coordinate of center of gravity
center = []
euclidean_list = []
for rect in contours_large:
    M = cv2.moments(rect)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    center.append([cx,cy])
center = np.array(center)
# calculate the nearest particle distance and fill them to euclidean_list
for c in center:
    ver_hori = np.sum(abs(center - c), axis=1)
    ver_hori[np.where(ver_hori == 0)] = 10000
    euclidean = np.linalg.norm(center[np.argmin(ver_hori)] - c)
    euclidean_list.append(round(euclidean, 4))

*pixelを数えると、contourAreaは近似しているため約0.825倍になる。*

In [44]:
all_list = []
all_list.append(labels_strain)
all_list.append(labels_orien)
all_list.append(roundnesses)
all_list.append(aspectos)
all_list.append(areas)
all_list.append(tilts)
all_list.append(euclidean_list)
all_list.append(f_or_b)

In [45]:
df = pd.DataFrame(all_list).T
df.rename(columns={0: 'strain', 1:'orien', 2:'roundness', 3:'aspect', 4:'size', 5:'tilt', \
                   6:'nearest_grain', 7:'ferrite_bainite'}, inplace=True)

In [46]:
print(df.shape)
df

(239, 8)


,strain,orien,roundness,aspect,size,tilt,nearest_grain,ferrite_bainite
0,3.0,2.0,0.657,0.839,299.0,19.243,47.2652,1.0
1,0.0,1.0,0.502,0.670,602.5,70.258,34.2053,1.0
2,1.0,2.0,0.471,0.369,423.0,47.945,32.5576,0.0
3,2.0,2.0,0.266,0.332,340.0,88.913,34.2053,1.0
4,2.0,3.0,0.326,0.506,282.0,41.532,51.5461,1.0
5,2.0,2.0,0.242,0.143,272.5,38.433,32.5576,0.0
6,0.0,1.0,0.438,0.721,638.5,9.140,47.2652,1.0
7,0.0,1.0,0.556,0.589,319.0,45.562,32.6956,1.0
8,2.0,3.0,0.136,0.812,672.0,3.881,29.7321,1.0
9,0.0,1.0,0.258,0.827,1754.5,87.171,47.2969,1.0


*scale fix*

In [47]:
# findContours function leads size to 0.825, so need to make it 1.175 time for only 1st-sample
# df['size'] = df['size']*1.175
df['size'] = df['size']*SCALE*SCALE

In [48]:
df.to_csv(str(ROOT_DIR / 'output' / 'csv_files' / 'df_4features.csv'), index=False)

In [22]:
import pandas as pd
df = pd.read_csv(str(ROOT_DIR / 'output' / 'csv_files' / 'df_4features.csv'))
df_strain4_size = pd.read_csv(str(ROOT_DIR / 'output' / 'csv_files' / 'df_strain4_size.csv'))
df = pd.concat([df, df_strain4_size], axis=1)
df.dropna(inplace=True)
df

,strain,orien,roundness,aspect,size,tilt,nearest_grain,ferrite_bainite,size4
0,0.0,1.0,0.537,0.477,0.522633,32.222,27.1662,0.0,0.637357
1,1.0,1.0,0.598,0.898,1.013398,26.152,14.7648,1.0,0.956035
2,1.0,3.0,0.476,0.429,0.669225,6.371,20.0998,0.0,0.675598
3,0.0,1.0,0.379,0.330,0.726587,8.566,27.1662,0.0,0.879553
4,1.0,3.0,0.433,0.301,0.956035,2.852,20.0998,0.0,0.879553
5,1.0,1.0,0.401,0.564,1.096254,27.389,32.8938,0.0,0.701093
6,1.0,2.0,0.449,0.404,0.892300,9.741,33.5261,0.0,0.726587
7,1.0,1.0,0.641,0.469,0.318678,40.252,34.1321,1.0,0.305931
8,2.0,1.0,0.457,0.879,0.943288,24.843,25.0599,0.0,0.509886
9,2.0,0.0,0.623,0.731,0.420656,1.684,11.0454,0.0,0.089230


In [23]:
# df['division'] = 100 - df['size4']/df['size']*100    #割り算
df['subtraction'] = df['size'] - df['size4']                #引き算
df

,strain,orien,roundness,aspect,size,tilt,nearest_grain,ferrite_bainite,size4,subtraction
0,0.0,1.0,0.537,0.477,0.522633,32.222,27.1662,0.0,0.637357,-0.114724
1,1.0,1.0,0.598,0.898,1.013398,26.152,14.7648,1.0,0.956035,0.057362
2,1.0,3.0,0.476,0.429,0.669225,6.371,20.0998,0.0,0.675598,-0.006374
3,0.0,1.0,0.379,0.330,0.726587,8.566,27.1662,0.0,0.879553,-0.152966
4,1.0,3.0,0.433,0.301,0.956035,2.852,20.0998,0.0,0.879553,0.076483
5,1.0,1.0,0.401,0.564,1.096254,27.389,32.8938,0.0,0.701093,0.395161
6,1.0,2.0,0.449,0.404,0.892300,9.741,33.5261,0.0,0.726587,0.165713
7,1.0,1.0,0.641,0.469,0.318678,40.252,34.1321,1.0,0.305931,0.012747
8,2.0,1.0,0.457,0.879,0.943288,24.843,25.0599,0.0,0.509886,0.433403
9,2.0,0.0,0.623,0.731,0.420656,1.684,11.0454,0.0,0.089230,0.331426


In [11]:
df.sort_values(by=['division'], ascending=True)
# df.sort_values(by=['subtraction'], ascending=True)

,strain,orien,roundness,aspect,size,tilt,nearest_grain,ferrite_bainite,size4,division
161,1.0,2.0,0.411,0.290,0.412131,84.114,41.0488,0.0,0.477719,-15.914489
45,1.0,2.0,0.229,0.285,1.106195,65.406,25.0000,0.0,1.133605,-2.477876
80,1.0,1.0,0.260,0.451,1.391064,67.031,33.9706,0.0,1.401833,-0.774103
91,1.0,1.0,0.422,0.396,0.611833,23.270,47.4342,0.0,0.612812,-0.160000
81,1.0,2.0,0.320,0.989,1.379317,56.352,27.2947,1.0,1.358759,1.490419
160,2.0,1.0,0.226,0.417,0.921176,68.173,58.3095,1.0,0.890829,3.294368
2,1.0,2.0,0.471,0.369,0.828178,47.945,32.5576,0.0,0.796852,3.782506
103,1.0,2.0,0.374,0.563,0.724411,63.146,47.4342,0.0,0.681338,5.945946
198,1.0,2.0,0.331,0.608,0.566802,50.578,31.1448,0.0,0.532540,6.044905
208,1.0,1.0,0.197,0.151,2.017582,36.979,31.1448,0.0,1.891299,6.259098


In [12]:
# division
df = df[df.division >= 0]
df.to_csv(str(ROOT_DIR / 'output' / 'csv_files' / 'df.csv'), index=False)

In [24]:
# subtraction
df = df[df.subtraction >= 0]
df.to_csv(str(ROOT_DIR / 'output' / 'csv_files' / 'df_subtraction.csv'), index=False)

In [25]:
df.shape

(315, 10)

*final process: integrate all csv from sample 1-3*

In [26]:
TEMP_DIR1 = Path().resolve() / const.sample1
TEMP_DIR2 = Path().resolve() / const.sample2
TEMP_DIR3 = Path().resolve() / const.sample3
# df1 = pd.read_csv(str(TEMP_DIR1 / 'output' / 'csv_files' / 'df.csv'))
# df2 = pd.read_csv(str(TEMP_DIR2 / 'output' / 'csv_files' / 'df.csv'))
# df3 = pd.read_csv(str(TEMP_DIR3 / 'output' / 'csv_files' / 'df.csv'))
df1 = pd.read_csv(str(TEMP_DIR1 / 'output' / 'csv_files' / 'df_subtraction.csv'))
df2 = pd.read_csv(str(TEMP_DIR2 / 'output' / 'csv_files' / 'df_subtraction.csv'))
df3 = pd.read_csv(str(TEMP_DIR3 / 'output' / 'csv_files' / 'df_subtraction.csv'))
df_all = pd.concat([df1, df2], axis=0)
df_all = pd.concat([df_all, df3], axis=0)
df_all.reset_index(drop=True, inplace=True)

In [27]:
df_all

,strain,orien,roundness,aspect,size,tilt,nearest_grain,ferrite_bainite,size4,subtraction
0,3.0,2.0,0.657,0.839,0.585402,19.243,47.2652,1.0,0.420941,0.164461
1,0.0,1.0,0.502,0.670,1.179615,70.258,34.2053,1.0,1.080742,0.098872
2,1.0,2.0,0.471,0.369,0.828178,47.945,32.5576,0.0,0.796852,0.031326
3,2.0,2.0,0.266,0.332,0.665675,88.913,34.2053,1.0,0.499256,0.166419
4,2.0,3.0,0.326,0.506,0.552118,41.532,51.5461,1.0,0.434646,0.117472
5,2.0,2.0,0.242,0.143,0.533519,38.433,32.5576,0.0,0.379826,0.153693
6,0.0,1.0,0.438,0.721,1.250098,9.140,47.2652,1.0,0.706790,0.543308
7,0.0,1.0,0.556,0.589,0.624559,45.562,32.6956,1.0,0.383742,0.240818
8,2.0,3.0,0.136,0.812,1.315686,3.881,29.7321,1.0,0.753779,0.561908
9,0.0,1.0,0.258,0.827,3.435077,87.171,47.2969,1.0,1.429243,2.005834


In [28]:
TEMP_DIR_ALL = Path().resolve() / 'all_sample'
# df_all.to_csv(str(TEMP_DIR_ALL / 'df_all.csv'), index=False)
df_all.to_csv(str(TEMP_DIR_ALL / 'df_all_subtraction.csv'), index=False)
# df_all = pd.read_csv(str(TEMP_DIR_ALL / 'df_all.csv'))

*calculate size for strain4%*